In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.firefox.options import Options
import time
import os
from fake_useragent import UserAgent
from random import randint
import pandas as pd
import numpy as np
import requests
import pandas as pd
import re
from selenium.webdriver.support.ui import Select
from dataclasses import dataclass, field

In [5]:
options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')
#opti#     driver = webdriver.Firefox()ons.add_argument("--headless")
driver = webdriver.Firefox(firefox_options=options)
driver.get('https://canvasy.net/product-category/%d9%84%d9%88%d8%ad%d8%a7%d8%aa-%d8%a7%d8%b2%d9%87%d8%a7%d8%b1/page/9/')

Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.14 (KHTML, like Gecko) Chrome/24.0.1292.0 Safari/537.14


In [129]:
driver.get('https://canvasy.net/product-category/%d9%84%d9%88%d8%ad%d8%a7%d8%aa-%d8%a7%d8%b2%d9%87%d8%a7%d8%b1/page/9/')

In [ ]:
urls = []

In [7]:
r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
soup = BeautifulSoup(r, "html.parser")


In [9]:
images = soup.find_all('div', {'class': 'product-image-thumbnail'})
len(images)
list_images = [img.find('img')['src'].replace('-300x300', '') for img in images]
base_image = list_images[0]
add_images = ','.join(list_images[1:])

In [10]:
list_images

['https://canvasy.net/wp-content/uploads/2020/08/كانفس-95.png',
 'https://canvasy.net/wp-content/uploads/2020/08/mockup-48.jpg',
 'https://canvasy.net/wp-content/uploads/2020/08/اسود-14.jpg',
 'https://canvasy.net/wp-content/uploads/2020/08/ذهبي-52.jpg',
 'https://canvasy.net/wp-content/uploads/2020/08/خشبي-58.jpg',
 'https://canvasy.net/wp-content/uploads/2020/08/فضي-53.jpg',
 'https://canvasy.net/wp-content/uploads/2020/08/flo0020-1.jpg']

In [133]:
def scrap_product(driver, prozes=None, size=None):
    r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
    soup = BeautifulSoup(r, "html.parser")
    name = soup.find('h1',{'class': 'product_title entry-title wd-entities-title'}).text.strip()
    try:
        prices = soup.find('div', {'class': 'single_variation_wrap'}).find_all('span', {'class': 'woocommerce-Price-amount amount'})
        try:
            price = prices[0].text.strip()
            special_price = prices[1].text.strip()
        except:
            price = prices[0].text.strip()
            special_price = ''
        
    except: 
        price =''
    description = soup.find('div', {'class': 'woocommerce-product-details__short-description'}).text.strip()
    cat1 = soup.find('a', {'class': 'breadcrumb-link breadcrumb-link-last'}).text.strip()
    sku = soup.find('span', {'class', 'sku'}).text.strip()
    try:
        qty = soup.find('p', {'class': 'stock in-stock'}).text.replace('متوفر في المخزون', '').strip()
    except:
        qty = ''
    images = soup.find_all('div', {'class': 'product-image-thumbnail'})
    len(images)
    list_images = [img.find('img')['src'].replace('-300x300', '') for img in images]
    base_image = list_images[0]
    add_images = ','.join(list_images[1:])
    data = {
        'sku': sku,
        'name': name,
        'price': price,
        'special_price': special_price,
        'link_url': driver.current_url,
        'qty': qty,
        'prozes': prozes,
        'size': size,
        'description': description,
        'base_image': base_image,
        'add_images': add_images,
        'cat1': cat1,
        'cat2': '',
        
    }
    return data


In [ ]:
df = pd.read_excel('cancasy_product_model.xlsx')
def try_except(name):
        try:
            return name.first_selected_option.text
        except:
            return None
        
for i, url in enumerate(urls):
    print('Count: ', i)
    print('URL: ', url)
    driver.get(url)
    time.sleep(1)
    prozes = driver.find_elements_by_xpath('//table[@class="variations"]//ul[@class="variable-items-wrapper image-variable-wrapper"]//li')
    
    
    #select = Select(driver.find_element_by_id('pa_retsting'))
    count = len(prozes)
    for t in prozes:
        time.sleep(1)
        t.click()
        print('Dans le prozes')
        proz = t.find_element_by_tag_name('img').get_attribute('alt')
        print('prozes', t.find_element_by_tag_name('img').get_attribute('alt'))
        sizes1 = driver.find_elements_by_xpath('//table[@class="variations"]//select')
        sizes_id = sizes1[1].get_attribute('id')
        sizes = driver.find_elements_by_xpath(f'//select[@id="{sizes_id}"]//option')
        select = Select(driver.find_element_by_id(sizes_id))
        for i in range(1, len(sizes)):
            time.sleep(1)
            select.select_by_index(i)
            size =try_except(select)
            print('Size: ', size)
            data = scrap_product(driver, prozes=proz, size=size)
            df1 = pd.DataFrame([data])
            df = pd.concat([df, df1], ignore_index=True)
            df.to_excel('cancasy_product_update.xlsx')
            

            

In [ ]:
sizes1 = driver.find_elements_by_xpath('//table[@class="variations"]//select')
sizes_id = sizes1[1].get_attribute('id')
print(sizes_id)
sizes = driver.find_elements_by_xpath(f'//select[@id="{sizes_id}"]//option')
select = Select(driver.find_element_by_id(sizes_id))
for i in range(1, len(sizes)):
        try:
            time.sleep(1)
       
            size =try_except(select)
            select.select_by_index(i)
            print('Size: ', size)
            data = scrap_product(driver, prozes=proz, size=size)
            df1 = pd.DataFrame([data])
            df = pd.concat([df, df1], ignore_index=True)
            df.to_excel('cancasy_product_update.xlsx')
        except:
            continue

In [ ]:
prozes = driver.find_elements_by_xpath('//table[@class="variations"]//ul[@class="variable-items-wrapper image-variable-wrapper"]//li')
len(prozes)
for t in prozes:
    t.click()
    print('prozes', t.find_element_by_tag_name('img').get_attribute('alt'))
    time.sleep(2)


In [ ]:
sizes = driver.find_elements_by_xpath('//select[@id="pa_retsting"]//option')
len(sizes)
select = Select(driver.find_element_by_id('pa_retsting'))
data = []
for i in range(1, len(sizes)):
    print('Count: ', i )
    select.select_by_index(i)
    size = try_except(select)
    print('Size: ', size)
    
    time.sleep(2)